In [ ]:
from pathlib import Path
import logging
from dotenv import dotenv_values
import folium

from generator_drainage_units.generator_order_levels import GeneratorOrderLevels

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
logging.basicConfig(level=logging.INFO)

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

In [ ]:
def run_generator_order_levels(
    path: Path,
    read_results: bool = True,
    write_results: bool = True
) -> GeneratorOrderLevels:
    
    order_generator = GeneratorOrderLevels(
        read_results=read_results, write_results=write_results
    )
    order_generator.read_data_from_case(path=path)
    
    return order_generator

In [ ]:
case_path = Path(base_dir, case_name)

order = run_generator_order_levels(path=case_path)

In [ ]:
def get_len_gdf(order):
    return len(order.hydroobjecten)

get_len_gdf(order)

In [ ]:
from shapely.geometry import Point
buffer_width = 0.5

lines = order.hydroobjecten[["CODE", "geometry"]].copy()
lines["start_node"] = lines.geometry.apply(lambda x: Point(x.coords[0][0:2]))
lines["end_node"] = lines.geometry.apply(lambda x: Point(x.coords[-1][0:2]))

In [ ]:
start_nodes = lines[["start_node"]].rename(columns={"start_node": "geometry"})
start_nodes['geometry'] = start_nodes.buffer(buffer_width)
end_nodes = lines[["end_node"]].rename(columns={"end_node": "geometry"})
end_nodes['geometry'] = end_nodes.buffer(buffer_width)

In [ ]:
outflow_points = start_nodes.sjoin(end_nodes, how="right")
outflow_points = lines.loc[outflow_points[outflow_points.index_left.isna()].index, ["end_node"]]
outflow_points = outflow_points.rename(columns={"end_node": "geometry"})

outflow_points_4326 = outflow_points.to_crs(4326)

m = folium.Map(
    location=[
        outflow_points_4326.geometry.y.mean(), 
        outflow_points_4326.geometry.x.mean()
    ], 
    zoom_start=13
)

folium.GeoJson(
    order.hydroobjecten.geometry,#.buffer(10),
    name="Watergangen",
    color="blue",
    fill_color="blue",
    zoom_on_click=True,
).add_to(m)

folium.GeoJson(
    outflow_points,
    name="Outflow points",
    marker=folium.Circle(radius=50, fill_color="orange", fill_opacity=0.4, color="orange", weight=3),
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
).add_to(m)

m
